In [2]:
import spacy
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score, precision_score, recall_score, classification_report
import os
from tqdm import tqdm
import conllu
import tqdm
import torch
import json
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import functional
from functools import partial
from transformers import BertModel
from torch import nn
from transformers import BertTokenizer
import os
import pandas as pd




In [3]:
def transform_to_binary(set):
    binary_labels = []
    for label in set:
        if label == "O":
            binary_labels.append(0)
        else:
            binary_labels.append(1)
    return binary_labels

In [4]:
## Conllu stuff
def filter_tags(x):
    return x        

def convert_to_list_dict(path, file, lower=None):
    path = path.format(file)
    if lower == True:
        with open(path, encoding="UTF-8") as infile:
            lst = []
            tokens = list(conllu.parse_incr(infile))
            for sent in tokens:
                    dic = {
                    "idx": sent.metadata["sent_id"],
                    "text": sent.metadata["text"].lower(),
                    "tokens": [token["form"].lower() for token in sent],
                    "lemmas": [token["lemma"] for token in sent],
                    "pos_tags": [token["upos"] for token in sent],
                    "ner_tags": [filter_tags(token["misc"].get("name", "O")) for token in sent],
                }
                    lst.append(dic) 
            print("Converting {} to list of dictionaries\n     {} elements converted..".format(file, len(lst)))
    else:
        with open(path, encoding="UTF-8") as infile:
            lst = []
            tokens = list(conllu.parse_incr(infile))
            for sent in tokens:
                    dic = {
                    "idx": sent.metadata["sent_id"],
                    "text": sent.metadata["text"],
                    "tokens": [token["form"] for token in sent],
                    "lemmas": [token["lemma"] for token in sent],
                    "pos_tags": [token["upos"] for token in sent],
                    "ner_tags": [filter_tags(token["misc"].get("name", "O")) for token in sent],
                }
                    lst.append(dic) 
            print("Converting {} to list of dictionaries\n     {} elements converted..".format(file, len(lst)))
    return lst

In [5]:
path = r"C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\Results\NCRFpp\all_conllu/{0}.conllu"
file_list = ["no_bokmaal-ud-dev", "no_bokmaal-ud-test", "no_bokmaal-ud-train", "no_nynorsk-ud-dev", "no_nynorsk-ud-test", "no_nynorsk-ud-train"]
test_split_no = convert_to_list_dict(path, file_list[1], lower=True)
test_split_ny = convert_to_list_dict(path, file_list[4], lower=True)
TRUE_test_split_no = convert_to_list_dict(path, file_list[1])
TRUE_test_split_ny = convert_to_list_dict(path, file_list[4])



test_split = test_split_no + test_split_ny
gold_split = TRUE_test_split_no + TRUE_test_split_ny



Converting no_bokmaal-ud-test to list of dictionaries
     1939 elements converted..
Converting no_nynorsk-ud-test to list of dictionaries
     1511 elements converted..
Converting no_bokmaal-ud-test to list of dictionaries
     1939 elements converted..
Converting no_nynorsk-ud-test to list of dictionaries
     1511 elements converted..


In [6]:
y_test_labels = [y["ner_tags"] for y in test_split]

In [7]:
nlp = spacy.load("nb_core_news_lg")



In [8]:
test = "Peter er hovedstaden i Norge"

doc = nlp(test)

for token in doc:
    print("{} {}-{}".format(token.text, token.ent_iob_, token.ent_type_))

Peter B-PER
er O-
hovedstaden O-
i O-
Norge B-GPE_LOC


In [9]:
pred_sentences = []
for sen in test_split:
    doc = nlp(" ".join(sen["tokens"]))
    labels = []
    for token in doc:
        if token.ent_iob_ != "O":
            labels.append("{}-{}".format(token.ent_iob_, token.ent_type_))
        else:
            labels.append("O")
    pred_sentences.append(labels)

In [10]:

for i, (pred, gold) in enumerate(zip(pred_sentences, y_test_labels)):
    if len(pred) != len(gold):
        print(f"{i}")

954
1063
1270
1813
2480
2554
2607
3112
3135
3138


In [11]:
drop_list = [954, 1063, 1270, 1813, 2480, 2554, 2607, 3112, 3135, 3138]

In [12]:
for i, n in enumerate(drop_list):
    pred_sentences.pop(n-i)
    y_test_labels.pop(n-i)

In [13]:
from seqeval.metrics import classification_report as cr
from seqeval.metrics import performance_measure
from seqeval.scheme import IOB2
from seqeval.metrics import performance_measure

In [14]:
def binary_parl_from_iob2(array):
    temp_lst = []
    for tag in array:
        if tag == "O":
            temp_lst.append(0)
        else:
            temp_lst.append(1)
    return temp_lst

In [15]:
y_pred = [item for sublist  in pred_sentences for item in sublist]
y_true = [item for sublist  in y_test_labels for item in sublist]

In [16]:
ner_vocab =     ['B-GPE_LOC', 'I-DRV', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 
                'I-GPE_ORG', 'B-GPE_ORG', 'B-EVT', 'B-DRV', 'I-PROD', 'B-ORG', 'B-MISC',
                'I-MISC', 'I-GPE_LOC', 'B-LOC', 'I-ORG', 'I-EVT', 'O', '@UNK']

In [17]:
gold_norne_binary=binary_parl_from_iob2(y_true)
preds_norne_binary=binary_parl_from_iob2(y_pred)

In [18]:
print(classification_report(gold_norne_binary, preds_norne_binary, digits=4))

              precision    recall  f1-score   support

           0     0.9662    0.9953    0.9805     51047
           1     0.8738    0.4840    0.6230      3446

    accuracy                         0.9629     54493
   macro avg     0.9200    0.7397    0.8017     54493
weighted avg     0.9603    0.9629    0.9579     54493



In [19]:
print(cr(y_test_labels, pred_sentences, mode='strict', scheme=IOB2, digits=4))


              precision    recall  f1-score   support

         DRV     0.4815    0.1667    0.2476        78
         EVT     0.0000    0.0000    0.0000        14
     GPE_LOC     0.7977    0.6464    0.7141       427
     GPE_ORG     0.4000    0.4590    0.4275        61
         LOC     0.5000    0.0865    0.1475       185
        MISC     0.0000    0.0000    0.0000        14
         ORG     0.6420    0.3179    0.4253       519
         PER     0.6028    0.3631    0.4532       953
        PROD     0.2941    0.0769    0.1220       130

   micro avg     0.6368    0.3587    0.4589      2381
   macro avg     0.4131    0.2352    0.2819      2381
weighted avg     0.6052    0.3587    0.4393      2381



c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
print(classification_report(y_true, y_pred, labels=ner_vocab[:-2], digits=4))

c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   B-GPE_LOC     0.7977    0.6464    0.7141       427
       I-DRV     1.0000    0.0769    0.1429        13
       I-LOC     0.7083    0.2000    0.3119        85
       B-PER     0.6794    0.4092    0.5108       953
       I-PER     0.8010    0.6680    0.7285       500
      B-PROD     0.3529    0.0923    0.1463       130
   I-GPE_ORG     0.5000    0.1429    0.2222         7
   B-GPE_ORG     0.4000    0.4590    0.4275        61
       B-EVT     0.0000    0.0000    0.0000        14
       B-DRV     0.4815    0.1667    0.2476        78
      I-PROD     0.5385    0.0556    0.1007       126
       B-ORG     0.6693    0.3314    0.4433       519
      B-MISC     0.0000    0.0000    0.0000        14
      I-MISC     0.0000    0.0000    0.0000         3
   I-GPE_LOC     0.9362    0.5570    0.6984        79
       B-LOC     0.5000    0.0865    0.1475       185
       I-ORG     0.7969    0.2056    0.3269       248
       I-EVT     0.0000    

c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# NPSC stuff

In [21]:
import json

class ParlamentaryCorpus():

    def __init__(self, path): # Takes input normalized_token_file
        self.path = path
        self.sentence_dict = {}

    def load_data(self, lower=False):
        sentence_dict = self.sentence_dict
        with open(self.path) as infile:
            with open(self.path, encoding="UTF-8") as infile:
                json_input = json.load(infile)

        if lower == False:
            for chaos in json_input["sentences"]:
                token_list = []
                for token in chaos["tokens"]:
                    if token["special_status"] != None:
                        continue
                    else:
                        if " " in token["token_text"]:
                            token_list.extend(token["token_text"].split())
                        else:
                            token_list.append(token["token_text"])
                    sentence_dict[chaos["sentence_id"]] = token_list
        else:
            for chaos in json_input["sentences"]:
                token_list = []
                for token in chaos["tokens"]:
                    if token["special_status"] != None:
                        continue
                    else:
                        if " " in token["token_text"]:
                            token_list.extend(token["token_text"].lower().split())
                        else:
                            token_list.append(token["token_text"].lower())
                    sentence_dict[chaos["sentence_id"]] = token_list
                    
        return sentence_dict

In [22]:
def load_parl_corpus(rootdir_parl_corpus, lower=False):
    corpora_normal_cap, corpora_lower, paths = [], [], []
    
    for subdir, dirs, files in os.walk(rootdir_parl_corpus):
        for file in files:
            if "normalized_token_data.json" in file:
                path = (os.path.join(subdir, file))
                paths.append(path)

    for corpus, path in enumerate(paths):
        corpus = ParlamentaryCorpus(path)
        corpus = corpus.load_data()
        corpora_normal_cap.append(corpus)
        for k, v in corpus.items():
            if v == []:
                print(k)
                print(path)

    if lower==True:
        for corpus, path in enumerate(paths):
            corpus = ParlamentaryCorpus(path)
            corpus = corpus.load_data(lower=True)
            corpora_lower.append(corpus)


    return corpora_normal_cap, corpora_lower

In [23]:
def gold_labels_unlabled_data(unlabeled):
    length = 0
    gold_labels = []
    for sentences in unlabeled:
        label = []
        length += len(sentences)
        for token in sentences:
            if token[0].isupper() == True: # Checks if token has captial letter
                label.append(1)
                
            else:
                label.append(0) # Adds 0 if token has lower 
        gold_labels.append(label)
    
    return gold_labels

In [24]:
def parl_sentences_only(dictionary_corpus):
    corp = []
    for diction in dictionary_corpus:
        corp += list(diction.values())
    return corp

In [25]:
""" Parliamentary corpus data loading stuff"""
nlp = spacy.load("nb_core_news_lg")
rootdir=r"C:\Users\Aarne\Desktop\Ferdig_code_folder\parl_corpus_full"
corpora_normal_cap, corpora_lower = load_parl_corpus(rootdir, lower=True)



In [26]:
gold_corp = parl_sentences_only(corpora_normal_cap)

_gold_labels_parl = gold_labels_unlabled_data(gold_corp)

In [27]:
pred_sentences_parl = []
for diction in (corpora_lower):
    for k, sentence in diction.items():
        doc = nlp(" ".join(sentence))
        labels = []
        for token in doc:
            if token.ent_iob_ != "O":
                labels.append("{}-{}".format(token.ent_iob_, token.ent_type_))
            else:
                labels.append("O")
        pred_sentences_parl.append(labels)

In [28]:
deep_copy_pred_sentences = pred_sentences_parl

In [29]:
pred_sentences_parl

[['B-ORG', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GPE_LOC'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'B-GPE_LOC'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'B-GPE_LOC', 'O', 'B-PER', 'I-PER', 'I-PER'],
 ['O', 'B-GPE_LOC', 'O', 'B-PER', 'I-PER'],
 ['O', 'B-GPE_LOC', 'O', 'B-PER', 'I-PER', 'I-PER'],
 ['O',
  'B-ORG',
  'O',
  'O',
  'O',
  'O',


In [30]:
parl_droplist = []
for i, (pred, gold) in enumerate(zip(pred_sentences_parl, _gold_labels_parl)):
    if len(pred) != len(gold):
        parl_droplist.append(i)

In [31]:
len(parl_droplist)

729

In [32]:
for i, n in enumerate(parl_droplist):
    pred_sentences_parl.pop(n-i)
    _gold_labels_parl.pop(n-i)

In [33]:
y_pred_parl = [item for sublist  in _gold_labels_parl for item in sublist]
y_true_parl = [item for sublist  in pred_sentences_parl for item in sublist]

In [34]:
gold_parl_binary=binary_parl_from_iob2(y_true_parl)

In [35]:
print(classification_report(gold_parl_binary, y_pred_parl, digits=4))

              precision    recall  f1-score   support

           0     0.9976    0.9828    0.9902   1013745
           1     0.5727    0.9077    0.7023     25723

    accuracy                         0.9810   1039468
   macro avg     0.7851    0.9452    0.8462   1039468
weighted avg     0.9871    0.9810    0.9830   1039468



In [36]:
"""LOADING NPSC SAMPLES """


import csv

with open(r'C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\IOB NPSC sample\200_annoterte_setninger.csv', newline='', encoding="UTF-8") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    left_col = []
    right_col = []
    for row in csvreader:
        left_col.append(row[0])
        right_col.append(row[1])

x, y = [], []

with open(r"C:\Users\Aarne\OneDrive - University of Bergen\Dokumenter\GitHub\MA_Peter-R-ysland-Aarnes\IOB NPSC sample\parl_annotation_comma_fixed 100 sents.csv", "r", encoding="UTF-8") as f_input:
    reader = csv.reader(f_input, delimiter=",")
    for i, row in enumerate(reader):
        if i%2:
            y.append(row)
        else:
            x.append(row)


def create_sublists(lst):
    sublists = []
    sublist = []
    for element in lst:
        if element == '':
            if sublist:
                sublists.append(sublist)
                sublist = []
        else:
            sublist.append(element)

    if sublist:
        sublists.append(sublist)

    return sublists

y200 = create_sublists(right_col)
x200 = create_sublists(left_col)


def lower_list_of_lists(lists):
    return [[word.lower() for word in sublist] for sublist in lists]


In [37]:
x = x200 + x

In [38]:
y_gold= y200 + y

In [39]:
test = lower_list_of_lists(x)

In [40]:
pred_sentences_parl_SAMPLES = []
for sentence in (test):
    doc = nlp(" ".join(sentence))
    labels = []
    for token in doc:
        if token.ent_iob_ != "O":
            labels.append("{}-{}".format(token.ent_iob_, token.ent_type_))
        else:
            labels.append("O")
    pred_sentences_parl_SAMPLES.append(labels)

In [41]:
drop_samples_samples = []
for i, (y, Y) in enumerate(zip(test, pred_sentences_parl_SAMPLES)):
    if len(y) != len(Y):
        drop_samples_samples.append(i)

In [42]:
len(drop_samples_samples)

3

In [43]:
for i, n in enumerate(drop_samples_samples):
    pred_sentences_parl_SAMPLES.pop(n-i)
    y_gold.pop(n-i)

In [44]:
print(cr(y_gold, pred_sentences_parl_SAMPLES, mode='strict', scheme=IOB2, digits=4))


              precision    recall  f1-score   support

         DRV     0.0000    0.0000    0.0000         3
         EVT     0.0000    0.0000    0.0000         0
     GPE_LOC     0.6970    0.8214    0.7541        28
     GPE_ORG     0.8333    0.2500    0.3846        20
         LOC     0.1667    0.5000    0.2500         2
         ORG     0.7111    0.4211    0.5289        76
         PER     0.7826    0.4186    0.5455        43
        PROD     0.5000    0.2000    0.2857         5

   micro avg     0.6780    0.4520    0.5424       177
   macro avg     0.4613    0.3264    0.3436       177
weighted avg     0.7159    0.4520    0.5333       177



c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
y_pred_SAMPLES = [item for sublist  in pred_sentences_parl_SAMPLES for item in sublist]
y_true_SAMPLES = [item for sublist  in y_gold for item in sublist]

In [46]:
print(classification_report(y_true_SAMPLES, y_pred_SAMPLES, digits=4, labels=ner_vocab[:-2]))

              precision    recall  f1-score   support

   B-GPE_LOC     0.6970    0.8214    0.7541        28
       I-DRV     0.0000    0.0000    0.0000         0
       I-LOC     0.0000    0.0000    0.0000         0
       B-PER     0.7826    0.4186    0.5455        43
       I-PER     0.9091    0.5000    0.6452        20
      B-PROD     0.5000    0.2000    0.2857         5
   I-GPE_ORG     0.0000    0.0000    0.0000         0
   B-GPE_ORG     0.8333    0.2500    0.3846        20
       B-EVT     0.0000    0.0000    0.0000         0
       B-DRV     0.0000    0.0000    0.0000         3
      I-PROD     0.0000    0.0000    0.0000         0
       B-ORG     0.7111    0.4211    0.5289        76
      B-MISC     0.0000    0.0000    0.0000         0
      I-MISC     0.0000    0.0000    0.0000         0
   I-GPE_LOC     1.0000    0.3333    0.5000         6
       B-LOC     0.1667    0.5000    0.2500         2
       I-ORG     0.0000    0.0000    0.0000        11
       I-EVT     0.0000    

c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aarne\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [47]:
gold_sample_binary=binary_parl_from_iob2(y_true_SAMPLES)
pred_sample_binary=binary_parl_from_iob2(y_pred_SAMPLES)

In [48]:
print(classification_report(gold_sample_binary, pred_sample_binary, digits=4))

              precision    recall  f1-score   support

           0     0.9818    0.9965    0.9891      5134
           1     0.8686    0.5561    0.6781       214

    accuracy                         0.9789      5348
   macro avg     0.9252    0.7763    0.8336      5348
weighted avg     0.9772    0.9789    0.9766      5348

